In [35]:
# automatically reload imported modules
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import sys
import matplotlib
from matplotlib import pyplot as plt
import pylab as pl

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [46]:
import helper

# FOLDER_PATH = 'F://term7//CV//reference'
FOLDER_PATH = 'D:\\ISTD\\Computer Vision\\Project\\ref2'
#FOLDER_PATH = 'F://term7//CV//ProjectTrail//A_manualLabel'
# FOLDER_PATH = 'C://Users//Dominic//Desktop//Concrete Crack Images for Classification'

# random
train_size = 600
test_size = 200
val_size = 50

img_range = np.arange(1, 1000)
X_train_pos_idx, X_test_pos_idx, X_val_pos_idx = helper.get_random_indices(img_range, train_size, test_size, val_size)
X_train_neg_idx, X_test_neg_idx, X_val_neg_idx = helper.get_random_indices(img_range, train_size, test_size, val_size)

X_train , Y_train = helper.get_concrete_data( X_train_pos_idx , X_train_neg_idx , path = FOLDER_PATH )
X_test  , Y_test  = helper.get_concrete_data( X_test_pos_idx  , X_test_neg_idx  , path = FOLDER_PATH )
X_val   , Y_val   = helper.get_concrete_data( X_val_pos_idx   , X_val_neg_idx   , path = FOLDER_PATH )

print( X_train.shape, Y_train.shape )
print( X_test.shape , Y_test.shape  )
print( X_val.shape  , Y_val.shape   )

(1200, 227, 227) (1200,)
(400, 227, 227) (400,)
(100, 227, 227) (100,)


# Load Filters

In [47]:
from filters import *
import preprocessing.shadow_reduction

filters = {}

grayscale       = GrayscaleNormalizer()
bilateral_canny = BilateralCanny()

filters['no filter']       = lambda x : x
filters['grayscale']       = grayscale.__call__
filters['canny']           = bilateral_canny.canny_img
filters['bilateral canny'] = bilateral_canny.bilateral_canny_img
filters['tophat']          = tophat
filters['canny gradient']  = canny_gradient
filters['otsu']            = otsu
filters['blackhat']        = blackhat
filters['gradient']        = gradient
filters['gradient2']       = gradient2
filters['gradient3']       = gradient3

In [48]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model

def get_models():
    models = {}
    
    models['clf']      = svm.SVC()
    models['neigh']    = KNeighborsClassifier(n_neighbors=3)
    models['regr']     = linear_model.LinearRegression()
    models['logistic'] = linear_model.LogisticRegression(C=1e5)
    
    return models

trained_models = {}

X_train = np.array(list(preprocessing.shadow_reduction.adaptiveThreshold(x) for x in X_train))

for filter_name, filter_func in filters.items():
    tmpX = np.array(list(filter_func(x) for x in X_train)).reshape(2*train_size, -1)
    
    sub_dict = {}
    
    for model_name, model in get_models().items():
        model.fit(tmpX, Y_train)
        
        sub_dict[model_name] = model
        
    trained_models[filter_name] = sub_dict

In [49]:
predict   = {}

for filter_name, filter_func in filters.items():
    tmpX = np.array(list(filter_func(x) for x in X_test)).reshape(2*test_size, -1)
    
    for model_name, model in trained_models[filter_name].items():
        result = model.predict(tmpX)

        predict["{0}, {1}".format(filter_name, model_name)] = (result[:test_size], result[test_size:])

In [50]:
for name, model in predict.items():
    print(name)
    
    if 'regr' in name:
        pos_count = 0
        neg_count = 0
        
        for i in predict[name][0]:
            if i > 0.5:
                pos_count +=1
        for i in predict[name][1]:
            if i > 0.5:
                neg_count +=1
                
    else:
        pos_count   = np.sum(predict[name][0])
        neg_count   = np.sum(predict[name][1])
        
    print("{0:>8.2f} / {1:<8.2f}".format(pos_count, test_size))
    print("{0:>8.2f} / {1:<8.2f}".format(test_size - neg_count, test_size))

no filter, clf
  200.00 / 200.00  
    0.00 / 200.00  
no filter, neigh
    0.00 / 200.00  
  200.00 / 200.00  
no filter, regr
  200.00 / 200.00  
    0.00 / 200.00  
no filter, logistic
    1.00 / 200.00  
  200.00 / 200.00  
grayscale, clf
  200.00 / 200.00  
    2.00 / 200.00  
grayscale, neigh
    0.00 / 200.00  
  200.00 / 200.00  
grayscale, regr
    4.00 / 200.00  
  200.00 / 200.00  
grayscale, logistic
    3.00 / 200.00  
  200.00 / 200.00  
canny, clf
  102.00 / 200.00  
  157.00 / 200.00  
canny, neigh
    0.00 / 200.00  
  200.00 / 200.00  
canny, regr
    0.00 / 200.00  
  200.00 / 200.00  
canny, logistic
   55.00 / 200.00  
  174.00 / 200.00  
bilateral canny, clf
   38.00 / 200.00  
  184.00 / 200.00  
bilateral canny, neigh
    0.00 / 200.00  
  200.00 / 200.00  
bilateral canny, regr
    0.00 / 200.00  
  200.00 / 200.00  
bilateral canny, logistic
   18.00 / 200.00  
  191.00 / 200.00  
tophat, clf
  117.00 / 200.00  
   80.00 / 200.00  
tophat, neigh
    0.00 / 200